<a href="https://colab.research.google.com/github/lzhkaya98/SSMGlobal/blob/main/Differential_SSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install torch

import torch
import torch.nn as nn
import torch.nn.functional as tf
from torch.utils import data
import numpy as np
import xlrd
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pylab

from google.colab import drive
drive.mount('/content/drive')

#数据集加载
sta_train , dyn_train, sigma_train, cos_train, rough_train, y1 , sta_test, dyn_test, sigma_test, cos_test, rough_test, y2 = load_data(str('/content/drive/My Drive/SMN-SDR.xls'))
#静态数据集，动态数据集(5个时间步)，反向散射系数集、入射角集，地表粗糙度信息集

#网络超参数设置
#FNN
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.bn = nn.BatchNorm1d(input_size)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.fc3 = nn.Linear(hidden_size,hidden_size)
        self.fc4 = nn.Linear(hidden_size,output_size)

    def forward(self,x):
        x = self.bn(x)
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = self.fc4(x)
        return x
#LSTM
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

#微分WCM模型
#动态参数VC，静态参数C,C=A/B,VC=B*vegetation correlation coefficient
def WCM(C, VC, sigma, cos_theta, rough):
    gamma = torch.exp(-2 * VC / cos_theta)
    sigma_veg = C * VC * cos_theta * (1 - gamma)
    sigma_soil = (sigma - sigma_veg) / gamma
    input = torch.cat([sigma_soil, rough], dim=1)
    return input

#散点图
def scatter_plot_frequency(x, y):
  xy = np.vstack([x,y])
  z = gaussian_kde(xy)(xy)
  # Sort the points by density, so that the densest points are plotted last
  idx = z.argsort()
  x, y, z = x[idx], y[idx], z[idx]
  fig, ax = plt.subplots()
  # 设置参考的1：1虚线参数
  xxx = [0, 50]  #[0, 0.6]
  yyy = [0, 50]
  plt.plot(xxx, yyy, c='0', linewidth=1, linestyle=':', marker='.', alpha=0.3)  # 绘制虚线
  plt.scatter(x, y, c=z, s=20, cmap='Spectral_r')
  z=np.polyfit(x,y,1)
  p=np.poly1d(z)
  pylab.plot(x,p(x),"r")
  plt.colorbar()
  plt.show()

def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=True)

#模型训练
batch_size = 256 #256
num_epochs = 6000  #6000
data_iter = load_array((dyn_train,sta_train,sigma_train,cos_train,rough_train,y1), batch_size)

model_C = Net(3,10,1)
model_VC = LSTM(4,20,2,1)
model_SM = Net(4,15,1)

#定义损失函数和优化器
criterion = nn.MSELoss()
trainer1 = torch.optim.SGD(model_SM.parameters(),lr=0.001)
trainer2 = torch.optim.SGD(model_VC.parameters(),lr=0.001)
trainer3 = torch.optim.SGD(model_C.parameters(),lr=0.001)

for epoch in range(num_epochs):
    for dyn,sta,sig,cost,rou,y in data_iter:
        VC = model_VC(dyn)
        C = model_C(sta)
        input = WCM(C, VC, sig, cost, rou)
        SM = model_SM(input)

        loss = criterion(SM, y)
#        print(loss)
        model_SM.zero_grad()
        model_VC.zero_grad()
        model_C.zero_grad()
        loss.backward()
        trainer1.step()
        trainer2.step()
        trainer3.step()
#    print(f'epoch {epoch+1}')

#模型保存
torch.save(model_C.state_dict(), str('/content/drive/My Drive/model_C.pth'))
torch.save(model_VC.state_dict(), str('/content/drive/My Drive/model_VC.pth'))
torch.save(model_SM.state_dict(), str('/content/drive/My Drive/model_SM.pth'))

#模型加载 需实例化模型model_C = Net(2,5,1)
#model_C.load_state_dict(torch.load(str('/content/drive/My Drive/model_C.pth')))
#model_VC.load_state_dict(torch.load(str('/content/drive/My Drive/model_VC.pth')))
#model_SM.load_state_dict(torch.load(str('/content/drive/My Drive/model_SM.pth')))
#model_C.eval();model_VC.eval();model_SM.eval()

#模型测试
VC_test = model_VC(dyn_test)
C_test = model_C(sta_test)
input_test = WCM(C_test, VC_test, sigma_test, cos_test, rough_test)
SM_test = model_SM(input_test)
loss = criterion(SM_test, y2)
print(loss)

#模型输出评估
#"""
from sklearn.metrics import r2_score
from scipy.stats import pearsonr

Y_test = y2.numpy()
Y_pred = SM_test.detach().numpy()
#print(Y_test)
#print(Y_pred)
r2 = r2_score(Y_test, Y_pred)
ResidualSquare = (Y_pred - Y_test) ** 2
MSE = np.mean(ResidualSquare)
bias = np.mean(Y_pred - Y_test)
RMSE = np.sqrt(MSE)
ubRMSE = np.sqrt(RMSE ** 2 - bias ** 2)
print(f'R2={r2}')
print(f'bias={bias}')
print(f'RMSE={RMSE}')
print(f'ubRMSE={ubRMSE}')
print("R P", pearsonr(Y_pred[:,0], Y_test[:,0]))
scatter_plot_frequency(Y_test[:,0],Y_pred[:,0])  #生成散点图
#"""